In [4]:
!pip3 install sparqlwrapper
!pip3 install pandas
!pip3 install names

     |################################| 231 kB 30.6 MB/s eta 0:00:01
     |################################| 41 kB 1.6 MB/s  eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |################################| 9.5 MB 18.1 MB/s eta 0:00:01
     |################################| 503 kB 30.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |################################| 789 kB 20.1 MB/s eta 0:00:01
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=805479 sha256=2353e39449e7904e683efc48cb78d3789c5f2010735ec62cd595fa8f06a2c875
  Stored in directory: /root/.cache/pip/wheels/bc/0a/39/6248e200b3403f09f9251d10b1cfef7b52e4c16135ae09a9ba
Successfully built names
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
sparql.setQuery("""
SELECT ?item ?itemLabel
WHERE
{
    ?item wdt:P31 wd:Q41487.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
""")

results = sparql.query().convert()

results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['item.value', 'itemLabel.value']].head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


"\n?country ?countryLabel ?region ?regionLabel\n?item wdt:P131 ?region.\n?item wdt:P17 ?country.\n'country.value', 'countryLabel.value', 'region.value', 'regionLabel.value'\n"

In [ ]:
sparql.setQuery("""
SELECT ?item ?itemLabel ?country ?countryLabel ?region ?regionLabel
WHERE
{
    ?item wdt:P31 wd:Q41487.
    ?item wdt:P131 ?region.
    ?item wdt:P17 ?country.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['item.value', 'itemLabel.value', 'country.value', 'countryLabel.value', 'region.value', 'regionLabel.value']].head()

In [42]:
ADDRESS = "http://www.wikidata.org/entity/{}"
sentences = ["The {ct} is located in {co}.", "The {ct} is in {co}.", "The {ct} activates in {co}.", "The location of the {ct} is {co}", "The location of the {ct} is in {co}"]
sentences.reverse()
i = 0
data = []
for row in results_df[['item.value', 'itemLabel.value', 'country.value', 'countryLabel.value', 'region.value', 'regionLabel.value']].iterrows():
    if i == 63 * 5:
        break
    if ADDRESS.format(row[1][1]) == row[1][0] or ADDRESS.format(row[1][5]) == row[1][4]:
        continue
    data.append(sentences[i // 63].format(ct=row[1][1], co=row[1][5]))
    i += 1

In [43]:
import pickle
with open("artificial_court_regions", "wb") as f:
    pickle.dump(data, f)

In [40]:
sentences.reverse()

In [44]:
data

['The location of the Koper District Court is in Koper',
 'The location of the Courthouses in Negotino is in Negotino Municipality',
 'The location of the Courthouses in Kratovo is in Kratovo Municipality',
 'The location of the Justizia Jauregia, Azpeitia is in Azpeitia',
 'The location of the Court of Milan is in Milan',
 'The location of the Tribunal da Relação de Guimarães is in Oliveira, São Paio e São Sebastião',
 'The location of the Tribunal de Execução de Penas de Coimbra is in Coimbra (Sé Nova, Santa Cruz, Almedina e São Bartolomeu)',
 'The location of the Tribunal da Relação de Coimbra is in Coimbra (Sé Nova, Santa Cruz, Almedina e São Bartolomeu)',
 "The location of the Lisbon's Administrative Court is in Parque das Nações",
 'The location of the Instalações Navais de Alcântara is in Estrela',
 'The location of the Landgericht Roth is in Roth',
 'The location of the Landgericht Greding is in Greding',
 'The location of the High Court of Botswana is in South-East District',


In [7]:
ADDRESS = "http://www.wikidata.org/entity/{}"
courts = []
for row in results_df[['item.value', 'itemLabel.value']].iterrows():
    if ADDRESS.format(row[1][1]) == row[1][0]:
        continue
    courts.append(row[1][1])

In [8]:
len(courts)

375

In [4]:
import names
for i in range(10):
    print(names.get_full_name())

Steve Labay
Karen Keys
Wallace Kelly
Theodora Platt
James Kelly
Joseph Willey
Jessica Conley
Breanna Urbanski
Roger Darden
Joseph Outlaw


In [35]:
import random

simple_sentences = []
for dec in ["accepted", "refused", "quashed", "dismissed", "allowed"]:
    for app in ["application", "appeal", "inquiry"]:
        r_courts = ["by the " + c for c in random.sample(courts, k=3)]
        for court in ["", " by the court"] + r_courts:
            i = 10
            for _ in range(i):
                last_name = random.choice(["Mr.", "Mrs."]) + " " + names.get_last_name()
                full_name = names.get_full_name()
                if random.choice(["Mr.", "Mrs."]) == "Mr.":
                    polite_full_name = "Mr. " + names.get_full_name(gender='male')
                else:
                    polite_full_name = "Mrs. " + names.get_full_name(gender='female')

                simple_sentences.append("The {app} by {name} was {dec}{court}.".format(app=app, name=last_name, dec=dec, court=court))
                simple_sentences.append("The {app} by {name} was {dec}{court}.".format(app=app, name=full_name, dec=dec, court=court))
                simple_sentences.append("The {app} by {name} was {dec}{court}.".format(app=app, name=polite_full_name, dec=dec, court=court))

In [62]:
import names

def random_name():
    name_type = random.choice(range(3))
    if name_type == 0:
        return random.choice(["Mr.", "Mrs."]) + " " + names.get_last_name()
    elif name_type == 1:
        return names.get_full_name()
    else:
        if random.choice(["Mr.", "Mrs."]) == "Mr.":
            return "Mr. " + names.get_full_name(gender='male')
        else:
            return "Mrs. " + names.get_full_name(gender='female')
        
def random_names(count):
    s = ""
    for i in range(count):
        if i == 0:
            s += random_name()
        elif i == count - 1:
            s += " and " + random_name()
        else:
            s += ", " + random_name()
            
    return s

ordering = ["first", "second", "third", "fourth", "fifth"]
def random_names_partition(count, partition, partition_by_name):
    s = ""
    r_names = []
    for i in range(count):
        r_name = random_name()
        r_names.append(r_name)
        if i == 0:
            s += r_name
        elif i == count - 1:
            s += " and " + r_name
        else:
            s += ", " + r_name
            
    first_group = sorted(random.sample(range(count), k=partition))
    second_group = list(range(count))
    s1 = "" if not partition_by_name else ("one by " if partition == 1 else "ones by ")
    if partition == 1:
        s1 += (r_names[first_group[0]] if partition_by_name else ordering[first_group[0]])
    else:
        for i, index in enumerate(first_group):
            second_group.remove(index)
            sub = (r_names[index] if partition_by_name else ordering[index])
            if i == 0:
                s1 += sub
            elif i == partition - 1:
                s1 += " and " + sub
            else:
                s1 += ", " + sub
            
    s2 = "" if not partition_by_name else ("one by " if count - partition == 1 else "ones by ")
    if count - partition == 1:
        s2 += (r_names[second_group[0]] if partition_by_name else ordering[second_group[0]])
    else:
        for i, index in enumerate(second_group):
            sub = (r_names[index] if partition_by_name else ordering[index])
            if i == 0:
                s2 += sub
            elif i == count - partition - 1:
                s2 += " and " + sub
            else:
                s2 += ", " + sub
            
    return s, s1, s2

In [19]:
medium_sentences = []
random_courts = courts + (["court"] * int(len(courts) * 0.05))
for dec in ["accepted", "refused", "dismissed", "allowed"]:
    for app in ["applications", "inquiries"]:
        for co in ["all", "none", "two", "three", "four"]:
            min_num = 3 if co == "two" else (4 if co == "three" else (5 if co == "four" else 2))
            for i in range(min_num, 6):
                num = 5
                for _ in range(num):
                    all_names = random_names(i)
                    court = random.choice(random_courts)
                    medium_sentences.append("The {court} received {app} by {names} and {dec} {co} of them".format(app=app, co=co, dec=dec, court=court, names=all_names))

In [36]:
len(simple_sentences)

2250

In [3]:
import pickle
with open("artificial_hard_applications", "rb") as f:
    s = pickle.load(f)
    print(len(s))
    print(s[0])

1120
The Court of Exchequer received applications by Anthony Durham and Salvador Dyess and accepted the one by Salvador Dyess, while refusing the one by Anthony Durham


In [66]:
import pickle
with open("artificial_hard_applications", "wb") as f:
    pickle.dump(hard_sentences, f)

In [63]:
hard_sentences = []
random_courts = courts + (["court"] * int(len(courts) * 0.05))
for dec in ["accepted", "allowed", "refused", "dismissed"]:
    while_l = ["accepting", "allowing"] if dec in ["refused", "dismissed"] else ["refusing", "dismissing"]
    for while_v in while_l:
        for app in ["applications", "inquiries"]:
            for i in range(2, 6):
                num = 5 * i
                for _ in range(num):
                    all_names, first, second = random_names_partition(i, partition=random.choice(range(1, i)), partition_by_name=random.choice([True, False]))
                    second = random.choices([second, "rest"], weights=[0.95, 0.05])[0]
                    court = random.choice(random_courts)
                    hard_sentences.append("The {court} received {app} by {names} and {dec} the {g1}, while {while_v} the {g2}".format(app=app, while_v=while_v, dec=dec, court=court, names=all_names, g1=first, g2=second))

In [64]:
len(hard_sentences)

1120

In [65]:
random.sample(hard_sentences, k=10)

['The Court of Jurisdictional Disputes received inquiries by Mrs. Millen, Dorothy Gartland, Mr. Gray and Mr. Christopher Tototzintle and dismissed the third, while accepting the first, second and third, fourth',
 'The Supreme Court of the Republic of Slovenia received applications by Kevin Ehlert, Mr. Collazo, David Tirado, Mrs. Jocelyn Mobley and Mrs. Mashburn and accepted the second and third, while refusing the first, fourth and fifth',
 'The Regional Federal Courts received inquiries by Mr. Angel Huhman, Mrs. Ruth Cullins, Mr. Shumpert and Mrs. Perham and refused the ones by Mr. Shumpert and Mrs. Perham, while accepting the ones by Mr. Angel Huhman and Mrs. Ruth Cullins',
 'The Amtsgericht Greding received inquiries by Mr. Brown, Mr. Wahba, Gabriela Stevenson and Mrs. Jenny Remus and allowed the first and fourth, while dismissing the second and third',
 'The Magistrates Court of Tasmania received inquiries by Gwendolyn Kositzke, Mrs. Rita Ceja, Mr. James Ferris, Mrs. Bernard and Ed